# Optimization with Linopy

In PyPSA `v0.22`, an additional optimization module was introduced to the package. It is built on [Linopy](https://github.com/PyPSA/linopy) and aims at 

* **performance** as we know it from the native PyPSA optimization (`lopf` with `pyomo=False`, now deprecated)
* **flexibility** as we know from the Pyomo implementation
* **usability** as we know from pandas/xarray


Linopy is a stand-alone package and works similar to Pyomo, but without the memory overhead and much faster. If you come from an older PyPSA version, you might be familiar with the pyomo-based ``n.lopf`` function. The Linopy-based optimization is the successor, is now the default optimization method in PyPSA, and is accessed via the [pypsa.Network.optimize](https://pypsa.readthedocs.io/en/latest/api/optimization.html) accessor. See the [migration guide](https://pypsa.readthedocs.io/en/stable/examples/optimization-with-linopy-migrate-extra-functionalities.html) for more information.

For additional information on the Linopy package, have a look at the [documentation](https://linopy.readthedocs.io/en/latest/).

## Let's get started

Now, we demonstrate the behaviour of the optimization with linopy. The core functions for the optimization can be called via the [pypsa.Network.optimize](https://pypsa.readthedocs.io/en/latest/api/optimization.html) accessor. The accessor is used for creating, solving, modifying the optimization problem. Further, it supports to run different optimization formulations and provides helper functions. 

At first, we run the ordinary linearized optimal power flow (LOPF). We then extend the formulation by some additional constraints.

In [1]:
import pypsa

In [2]:
n = pypsa.examples.ac_dc_meshed(from_master=True)

INFO:pypsa.io:Imported network ac-dc-meshed.nc has buses, carriers, generators, global_constraints, lines, links, loads


In order to make the network a bit more interesting, we modify its data: We set gas generators to non-extendable,

In [3]:
n.generators.loc[n.generators.carrier == "gas", "p_nom_extendable"] = False

... add ramp limits,

In [4]:
n.generators.loc[n.generators.carrier == "gas", "ramp_limit_down"] = 0.2
n.generators.loc[n.generators.carrier == "gas", "ramp_limit_up"] = 0.2

... add additional storage units (cyclic and non-cyclic) and fix one state_of_charge,

In [5]:
n.add(
    "StorageUnit",
    "su",
    bus="Manchester",
    marginal_cost=10,
    inflow=50,
    p_nom_extendable=True,
    capital_cost=10,
    p_nom=2000,
    efficiency_dispatch=0.5,
    cyclic_state_of_charge=True,
    state_of_charge_initial=1000,
)

n.add(
    "StorageUnit",
    "su2",
    bus="Manchester",
    marginal_cost=10,
    p_nom_extendable=True,
    capital_cost=50,
    p_nom=2000,
    efficiency_dispatch=0.5,
    carrier="gas",
    cyclic_state_of_charge=False,
    state_of_charge_initial=1000,
)

n.storage_units_t.state_of_charge_set.loc[n.snapshots[7], "su"] = 100

...and add an additional store.

In [6]:
n.add("Bus", "storebus", carrier="hydro", x=-5, y=55)
n.add(
    "Link",
    ["battery_power", "battery_discharge"],
    "",
    bus0=["Manchester", "storebus"],
    bus1=["storebus", "Manchester"],
    p_nom=100,
    efficiency=0.9,
    p_nom_extendable=True,
    p_nom_max=1000,
)
n.add(
    "Store",
    ["store"],
    bus="storebus",
    e_nom=2000,
    e_nom_extendable=True,
    marginal_cost=10,
    capital_cost=10,
    e_nom_max=5000,
    e_initial=100,
    e_cyclic=True,
);

## Run Optimization

The optimization based on linopy mimics the well-known [`n.lopf`](https://pypsa.readthedocs.io/en/v0.28.0/api_reference.html#pypsa.Network.lopf) optimization. We run it by calling the `optimize` accessor.

In [7]:
n.optimize()

Index(['Norwich Converter', 'Norway Converter', 'Bremen Converter', 'DC link',
       'battery_power', 'battery_discharge'],
      dtype='object', name='Link')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
Index(['store'], dtype='object', name='Store')
Index(['London', 'Norwich', 'Norwich DC', 'Manchester', 'Bremen', 'Bremen DC',
       'Frankfurt', 'Norway', 'Norway DC', 'storebus'],
      dtype='object', name='Bus')
Index(['Norwich Converter', 'Norway Converter', 'Bremen Converter', 'DC link',
       'battery_power', 'battery_discharge'],
      dtype='object', name='Link')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
Index(['store'], dtype='object', name='Store')
Index(['London', 'Norwich', 'Norwich DC', 'Manchester',

('ok', 'optimal')

We now have a model instance attached to our network. It is a container of all variables, constraints and the objective function. You can modify this as much as you please, by directly adding or deleting variables or constraints etc.

In [8]:
n.model

Linopy LP model

Variables:
----------
 * Generator-p_nom (Generator-ext)
 * Line-s_nom (Line-ext)
 * Link-p_nom (Link-ext)
 * Store-e_nom (Store-ext)
 * StorageUnit-p_nom (StorageUnit-ext)
 * Generator-p (snapshot, Generator)
 * Line-s (snapshot, Line)
 * Link-p (snapshot, Link)
 * Store-e (snapshot, Store)
 * StorageUnit-p_dispatch (snapshot, StorageUnit)
 * StorageUnit-p_store (snapshot, StorageUnit)
 * StorageUnit-state_of_charge (snapshot, StorageUnit)
 * StorageUnit-spill (snapshot, StorageUnit)
 * Store-p (snapshot, Store)
 * objective_constant

Constraints:
------------
 * Generator-ext-p_nom-lower (Generator-ext)
 * Generator-ext-p_nom-upper (Generator-ext)
 * Line-ext-s_nom-lower (Line-ext)
 * Line-ext-s_nom-upper (Line-ext)
 * Link-ext-p_nom-lower (Link-ext)
 * Link-ext-p_nom-upper (Link-ext)
 * Store-ext-e_nom-lower (Store-ext)
 * Store-ext-e_nom-upper (Store-ext)
 * StorageUnit-ext-p_nom-lower (StorageUnit-ext)
 * StorageUnit-ext-p_nom-upper (StorageUnit-ext)
 * Generator-

## Modify model, optimize and feed back to network 

When you have a fresh network and you just want to create the model instance, run

In [9]:
n.optimize.create_model()

Index(['0', '1', '2'], dtype='object', name='SubNetwork')
Index(['Norwich Converter', 'Norway Converter', 'Bremen Converter', 'DC link',
       'battery_power', 'battery_discharge'],
      dtype='object', name='Link')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
Index(['store'], dtype='object', name='Store')
Index(['London', 'Norwich', 'Norwich DC', 'Manchester', 'Bremen', 'Bremen DC',
       'Frankfurt', 'Norway', 'Norway DC', 'storebus'],
      dtype='object', name='Bus')
C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\linopy\common.py:147: UserWarning: coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.
  warn(


Linopy LP model

Variables:
----------
 * Generator-p_nom (Generator-ext)
 * Line-s_nom (Line-ext)
 * Link-p_nom (Link-ext)
 * Store-e_nom (Store-ext)
 * StorageUnit-p_nom (StorageUnit-ext)
 * Generator-p (snapshot, Generator)
 * Line-s (snapshot, Line)
 * Link-p (snapshot, Link)
 * Store-e (snapshot, Store)
 * StorageUnit-p_dispatch (snapshot, StorageUnit)
 * StorageUnit-p_store (snapshot, StorageUnit)
 * StorageUnit-state_of_charge (snapshot, StorageUnit)
 * StorageUnit-spill (snapshot, StorageUnit)
 * Store-p (snapshot, Store)
 * objective_constant

Constraints:
------------
 * Generator-ext-p_nom-lower (Generator-ext)
 * Generator-ext-p_nom-upper (Generator-ext)
 * Line-ext-s_nom-lower (Line-ext)
 * Line-ext-s_nom-upper (Line-ext)
 * Link-ext-p_nom-lower (Link-ext)
 * Link-ext-p_nom-upper (Link-ext)
 * Store-ext-e_nom-lower (Store-ext)
 * Store-ext-e_nom-upper (Store-ext)
 * StorageUnit-ext-p_nom-lower (StorageUnit-ext)
 * StorageUnit-ext-p_nom-upper (StorageUnit-ext)
 * Generator-

Through the model instance we gain a lot of flexibility. Let's say for example we want to remove the Kirchhoff Voltage Law constraint, thus convert the model to a transport model. This can be done via

In [10]:
n.model.constraints.remove("Kirchhoff-Voltage-Law")

Now, we want to optimize the altered model and feed to solution back to the network. Here again, we use the `optimize` accessor.

In [11]:
n.optimize.solve_model()

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.13s
INFO:linopy.solvers:Log file at C:\Users\dell\AppData\Local\Temp\highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 300 primals, 734 duals
Objective: 1.41e+07
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Generator-fix-p-ramp_limit_up, Generator-fix-p-ramp_limit_down, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-state_of_charge_set, StorageUnit-energy_balance, Store-energy_balance were not assigned 

('ok', 'optimal')

Here, we followed the recommended way to run altered models:

1. **Create the model instance** - `n.optimize.create_model()`
2. **Modify the model to your needs**
3. **Solve and feed back** - `n.optimize.solve_model()`


For compatibility reasons the `optimize` function, also allows passing a `extra_funcionality` argument, as we know it from the `lopf` function. The above behaviour with use of the extra functionality is obtained through

In [12]:
def remove_kvl(n, sns):
    print("KVL removed!")
    n.model.constraints.remove("Kirchhoff-Voltage-Law")


n.optimize(extra_functionality=remove_kvl)

Index(['0', '1', '2'], dtype='object', name='SubNetwork')
Index(['Norwich Converter', 'Norway Converter', 'Bremen Converter', 'DC link',
       'battery_power', 'battery_discharge'],
      dtype='object', name='Link')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
Index(['store'], dtype='object', name='Store')
Index(['London', 'Norwich', 'Norwich DC', 'Manchester', 'Bremen', 'Bremen DC',
       'Frankfurt', 'Norway', 'Norway DC', 'storebus'],
      dtype='object', name='Bus')
Index(['0', '1', '2'], dtype='object', name='SubNetwork')
Index(['Norwich Converter', 'Norway Converter', 'Bremen Converter', 'DC link',
       'battery_power', 'battery_discharge'],
      dtype='object', name='Link')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object',

KVL removed!


INFO:linopy.io: Writing time: 0.14s
INFO:linopy.solvers:Log file at C:\Users\dell\AppData\Local\Temp\highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 300 primals, 734 duals
Objective: 1.41e+07
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Generator-fix-p-ramp_limit_up, Generator-fix-p-ramp_limit_down, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-state_of_charge_set, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

## Additional constraints

In the following, we exemplarily present a set of additional constraints. Note, the dual values of the additional constraints won't be stored in default data fields in the `PyPSA` network. But in any case, they are stored in the `linopy.Model`. 

Again, we **first build** the optimization model, **add our constraints** and finally **solve the network**. For the first step, we use again our accessor `optimize` to access the function `create_model`. This returns the `linopy` model that we can modify.

In [13]:
m = n.optimize.create_model()  # the return value is the model, let's use it directly!

Index(['0', '1', '2'], dtype='object', name='SubNetwork')
Index(['Norwich Converter', 'Norway Converter', 'Bremen Converter', 'DC link',
       'battery_power', 'battery_discharge'],
      dtype='object', name='Link')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
Index(['store'], dtype='object', name='Store')
Index(['London', 'Norwich', 'Norwich DC', 'Manchester', 'Bremen', 'Bremen DC',
       'Frankfurt', 'Norway', 'Norway DC', 'storebus'],
      dtype='object', name='Bus')
C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\linopy\common.py:147: UserWarning: coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.
  warn(


1. **Minimum for state of charge**

Assume we want to set a minimum state of charge of 50 MWh in our storage unit. This is done by: 

In [14]:
sus = m.variables["StorageUnit-state_of_charge"]
m.add_constraints(sus >= 50, name="StorageUnit-minimum_soc")

Constraint `StorageUnit-minimum_soc` (snapshot: 10, StorageUnit: 2):
--------------------------------------------------------------------
[2015-01-01 00:00:00, su]: +1 StorageUnit-state_of_charge[2015-01-01 00:00:00, su]   ≥ 50.0
[2015-01-01 00:00:00, su2]: +1 StorageUnit-state_of_charge[2015-01-01 00:00:00, su2] ≥ 50.0
[2015-01-01 01:00:00, su]: +1 StorageUnit-state_of_charge[2015-01-01 01:00:00, su]   ≥ 50.0
[2015-01-01 01:00:00, su2]: +1 StorageUnit-state_of_charge[2015-01-01 01:00:00, su2] ≥ 50.0
[2015-01-01 02:00:00, su]: +1 StorageUnit-state_of_charge[2015-01-01 02:00:00, su]   ≥ 50.0
[2015-01-01 02:00:00, su2]: +1 StorageUnit-state_of_charge[2015-01-01 02:00:00, su2] ≥ 50.0
[2015-01-01 03:00:00, su]: +1 StorageUnit-state_of_charge[2015-01-01 03:00:00, su]   ≥ 50.0
		...
[2015-01-01 06:00:00, su2]: +1 StorageUnit-state_of_charge[2015-01-01 06:00:00, su2] ≥ 50.0
[2015-01-01 07:00:00, su]: +1 StorageUnit-state_of_charge[2015-01-01 07:00:00, su]   ≥ 50.0
[2015-01-01 07:00:00, su2]: 

The return value of the `add_constraints` function is a array with the labels of the constraints. You can access the constraint now through: 

In [15]:
m.constraints["StorageUnit-minimum_soc"]

Constraint `StorageUnit-minimum_soc` (snapshot: 10, StorageUnit: 2):
--------------------------------------------------------------------
[2015-01-01 00:00:00, su]: +1 StorageUnit-state_of_charge[2015-01-01 00:00:00, su]   ≥ 50.0
[2015-01-01 00:00:00, su2]: +1 StorageUnit-state_of_charge[2015-01-01 00:00:00, su2] ≥ 50.0
[2015-01-01 01:00:00, su]: +1 StorageUnit-state_of_charge[2015-01-01 01:00:00, su]   ≥ 50.0
[2015-01-01 01:00:00, su2]: +1 StorageUnit-state_of_charge[2015-01-01 01:00:00, su2] ≥ 50.0
[2015-01-01 02:00:00, su]: +1 StorageUnit-state_of_charge[2015-01-01 02:00:00, su]   ≥ 50.0
[2015-01-01 02:00:00, su2]: +1 StorageUnit-state_of_charge[2015-01-01 02:00:00, su2] ≥ 50.0
[2015-01-01 03:00:00, su]: +1 StorageUnit-state_of_charge[2015-01-01 03:00:00, su]   ≥ 50.0
		...
[2015-01-01 06:00:00, su2]: +1 StorageUnit-state_of_charge[2015-01-01 06:00:00, su2] ≥ 50.0
[2015-01-01 07:00:00, su]: +1 StorageUnit-state_of_charge[2015-01-01 07:00:00, su]   ≥ 50.0
[2015-01-01 07:00:00, su2]: 

and inspects its attributes like `lhs`, `sign` and `rhs`, e.g.

In [16]:
m.constraints["StorageUnit-minimum_soc"].rhs

<xarray.DataArray 'rhs' (snapshot: 10, StorageUnit: 2)> Size: 160B
array([[50., 50.],
       [50., 50.],
       [50., 50.],
       [50., 50.],
       [50., 50.],
       [50., 50.],
       [50., 50.],
       [50., 50.],
       [50., 50.],
       [50., 50.]])
Coordinates:
  * snapshot     (snapshot) datetime64[ns] 80B 2015-01-01 ... 2015-01-01T09:0...
  * StorageUnit  (StorageUnit) object 16B 'su' 'su2'

2. **Fix the ratio between ingoing and outgoing capacity of the Store**

The battery in our system is modelled with two links and a store. We should make sure that its charging and discharging capacities, meaning their links, are somehow coupled. 

In [17]:
capacity = m.variables["Link-p_nom"]
eff = n.links.at["battery_power", "efficiency"]
lhs = capacity.loc["battery_power"] - eff * capacity.loc["battery_discharge"]
m.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

Constraint `Link-battery_fix_ratio`
-----------------------------------
+1 Link-p_nom[battery_power] - 0.9 Link-p_nom[battery_discharge] = -0.0

3. **Every bus must in total produce the 20% of the total demand**

For this, we use the linopy function `groupby_sum` which follows the pattern from `pandas`/`xarray`'s `groupby` function.

In [18]:
total_demand = n.loads_t.p_set.sum().sum()
buses = n.generators.bus.to_xarray()
prod_per_bus = m.variables["Generator-p"].groupby(buses).sum().sum("snapshot")
m.add_constraints(prod_per_bus >= total_demand / 5, name="Bus-minimum_production_share")

Constraint `Bus-minimum_production_share` (bus: 3):
---------------------------------------------------
[Frankfurt]: +1 Generator-p[2015-01-01 00:00:00, Frankfurt Wind] + 1 Generator-p[2015-01-01 01:00:00, Frankfurt Wind] + 1 Generator-p[2015-01-01 02:00:00, Frankfurt Wind] ... +1 Generator-p[2015-01-01 07:00:00, Frankfurt Gas] + 1 Generator-p[2015-01-01 08:00:00, Frankfurt Gas] + 1 Generator-p[2015-01-01 09:00:00, Frankfurt Gas]        ≥ 6509.525616820881
[Manchester]: +1 Generator-p[2015-01-01 00:00:00, Manchester Wind] + 1 Generator-p[2015-01-01 01:00:00, Manchester Wind] + 1 Generator-p[2015-01-01 02:00:00, Manchester Wind] ... +1 Generator-p[2015-01-01 07:00:00, Manchester Gas] + 1 Generator-p[2015-01-01 08:00:00, Manchester Gas] + 1 Generator-p[2015-01-01 09:00:00, Manchester Gas] ≥ 6509.525616820881
[Norway]: +1 Generator-p[2015-01-01 00:00:00, Norway Wind] + 1 Generator-p[2015-01-01 01:00:00, Norway Wind] + 1 Generator-p[2015-01-01 02:00:00, Norway Wind] ... +1 Generator-p[2015

In [19]:
con = prod_per_bus >= total_demand / 5

In [20]:
con

Constraint (unassigned) (bus: 3):
---------------------------------
[Frankfurt]: +1 Generator-p[2015-01-01 00:00:00, Frankfurt Wind] + 1 Generator-p[2015-01-01 01:00:00, Frankfurt Wind] + 1 Generator-p[2015-01-01 02:00:00, Frankfurt Wind] ... +1 Generator-p[2015-01-01 07:00:00, Frankfurt Gas] + 1 Generator-p[2015-01-01 08:00:00, Frankfurt Gas] + 1 Generator-p[2015-01-01 09:00:00, Frankfurt Gas]        ≥ 6509.525616820881
[Manchester]: +1 Generator-p[2015-01-01 00:00:00, Manchester Wind] + 1 Generator-p[2015-01-01 01:00:00, Manchester Wind] + 1 Generator-p[2015-01-01 02:00:00, Manchester Wind] ... +1 Generator-p[2015-01-01 07:00:00, Manchester Gas] + 1 Generator-p[2015-01-01 08:00:00, Manchester Gas] + 1 Generator-p[2015-01-01 09:00:00, Manchester Gas] ≥ 6509.525616820881
[Norway]: +1 Generator-p[2015-01-01 00:00:00, Norway Wind] + 1 Generator-p[2015-01-01 01:00:00, Norway Wind] + 1 Generator-p[2015-01-01 02:00:00, Norway Wind] ... +1 Generator-p[2015-01-01 07:00:00, Norway Gas] + 1 Gen

... and now let's solve the network again. 

In [21]:
n.optimize.solve_model()

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.12s
INFO:linopy.solvers:Log file at C:\Users\dell\AppData\Local\Temp\highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 300 primals, 778 duals
Objective: 1.43e+07
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Generator-fix-p-ramp_limit_up, Generator-fix-p-ramp_limit_down, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-state_of_charge_set, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_bal

('ok', 'optimal')

## Analysing the constraints

Let's see if the system got our own constraints. We look at `n.constraints` which combines summarises constraints going into the linear problem

In [22]:
n.model.constraints

linopy.model.Constraints
------------------------
 * Generator-ext-p_nom-lower (Generator-ext)
 * Generator-ext-p_nom-upper (Generator-ext)
 * Line-ext-s_nom-lower (Line-ext)
 * Line-ext-s_nom-upper (Line-ext)
 * Link-ext-p_nom-lower (Link-ext)
 * Link-ext-p_nom-upper (Link-ext)
 * Store-ext-e_nom-lower (Store-ext)
 * Store-ext-e_nom-upper (Store-ext)
 * StorageUnit-ext-p_nom-lower (StorageUnit-ext)
 * StorageUnit-ext-p_nom-upper (StorageUnit-ext)
 * Generator-fix-p-lower (snapshot, Generator-fix)
 * Generator-fix-p-upper (snapshot, Generator-fix)
 * Generator-ext-p-lower (snapshot, Generator-ext)
 * Generator-ext-p-upper (snapshot, Generator-ext)
 * Generator-fix-p-ramp_limit_up (snapshot, Generator-fix)
 * Generator-fix-p-ramp_limit_down (snapshot, Generator-fix)
 * Line-ext-s-lower (snapshot, Line-ext)
 * Line-ext-s-upper (snapshot, Line-ext)
 * Link-ext-p-lower (snapshot, Link-ext)
 * Link-ext-p-upper (snapshot, Link-ext)
 * Store-ext-e-lower (snapshot, Store-ext)
 * Store-ext-e-up

The last three entries show our constraints. Let's check whether out two custom constraint are fulfilled:

In [23]:
n.links.loc[["battery_power", "battery_discharge"], ["p_nom_opt"]]

,p_nom_opt
Link,
battery_power,900.0
battery_discharge,1000.0


In [24]:
n.storage_units_t.state_of_charge

StorageUnit,su,su2
snapshot,,
2015-01-01 00:00:00,1835.745612,1000.000000
2015-01-01 01:00:00,1836.063354,490.091300
2015-01-01 02:00:00,1886.063354,490.091300
2015-01-01 03:00:00,1936.063354,490.091300
2015-01-01 04:00:00,1986.063354,1000.000000
2015-01-01 05:00:00,50.000000,50.000000
2015-01-01 06:00:00,50.000000,50.000000
2015-01-01 07:00:00,100.000000,156.725142
2015-01-01 08:00:00,150.000000,50.000000


In [25]:
n.generators_t.p.groupby(n.generators.bus, axis=1).sum().sum() / n.loads_t.p.sum().sum()

C:\Users\dell\AppData\Local\Temp\ipykernel_19560\2985613193.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  n.generators_t.p.groupby(n.generators.bus, axis=1).sum().sum() / n.loads_t.p.sum().sum()


bus
Frankfurt     0.200000
Manchester    0.200000
Norway        0.637048
dtype: float64

Looks good! Now, let's see which dual values were parsed. Therefore we have a look into `n.model.dual` 


In [26]:
n.model.dual

C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\linopy\common.py:400: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn(


<xarray.Dataset> Size: 7kB
Dimensions:                                (Generator-ext: 3, Line-ext: 7,
                                            Link-ext: 6, Store-ext: 1,
                                            StorageUnit-ext: 2, snapshot: 10,
                                            Generator-fix: 3,
                                            StorageUnit-state_of_charge_set_i: 1,
                                            Bus: 10, cycles: 2, StorageUnit: 2,
                                            Store: 1, Generator: 6, bus: 3)
Coordinates: (12/14)
  * Generator-ext                          (Generator-ext) object 24B 'Manche...
  * Line-ext                               (Line-ext) object 56B '0' '1' ... '6'
  * Link-ext                               (Link-ext) object 48B 'Norwich Con...
  * Store-ext                              (Store-ext) object 8B 'store'
  * StorageUnit-ext                        (StorageUnit-ext) object 16B 'su' ...
  * snapshot                               (snapshot) datetime64[ns] 80B 2015...
    ...                                     ...
  * Bus                                    (Bus) object 80B 'London' ... 'sto...
  * cycles                                 (cycles) int32 8B 0 1
  * StorageUnit                            (StorageUnit) object 16B 'su' 'su2'
  * Store                                  (Store) object 8B 'store'
  * Generator                              (Generator) object 48B 'Manchester...
  * bus                                    (bus) object 24B 'Frankfurt' ... '...
Data variables: (12/39)
    Generator-ext-p_nom-lower              (Generator-ext) float64 24B -0.0 ....
    Generator-ext-p_nom-upper              (Generator-ext) float64 24B nan .....
    Line-ext-s_nom-lower                   (Line-ext) float64 56B -0.0 ... -0.0
    Line-ext-s_nom-upper                   (Line-ext) float64 56B nan ... nan
    Link-ext-p_nom-lower                   (Link-ext) float64 48B -0.0 ... -0.0
    Link-ext-p_nom-upper                   (Link-ext) float64 48B nan ... -527.1
    ...                                     ...
    Generator-e_sum_min                    (Generator) float64 48B nan ... nan
    Generator-e_sum_max                    (Generator) float64 48B -0.0 ... -0.0
    GlobalConstraint-co2_limit             float64 8B -950.5
    StorageUnit-minimum_soc                (snapshot, StorageUnit) float64 160B ...
    Link-battery_fix_ratio                 float64 8B -567.9
    Bus-minimum_production_share           (bus) float64 24B 35.2 45.42 -0.0

In [27]:
n.model.dual["StorageUnit-minimum_soc"]

C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\linopy\common.py:400: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn(


<xarray.DataArray 'StorageUnit-minimum_soc' (snapshot: 10, StorageUnit: 2)> Size: 160B
array([[ -0.        ,  -0.        ],
       [ -0.        ,  -0.        ],
       [ -0.        ,  -0.        ],
       [ -0.        ,  -0.        ],
       [ -0.        ,  -0.        ],
       [ -0.        ,  -0.        ],
       [157.42265847,   4.44444444],
       [ -0.        ,  -0.        ],
       [ -0.        ,  -0.        ],
       [  5.55555556,  67.84841829]])
Coordinates:
  * snapshot     (snapshot) datetime64[ns] 80B 2015-01-01 ... 2015-01-01T09:0...
  * StorageUnit  (StorageUnit) object 16B 'su' 'su2'

In [ ]:
n.model.dual["Link-battery_fix_ratio"]

In [ ]:
n.model.dual["Bus-minimum_production_share"]

These are the basic functionalities of the `optimize` accessor. There are many more functions like abstract optimziation formulations (security constraint optimization, iterative transmission expansion optimization, etc.) or helper functions (fixing optimized capacities, adding load shedding). Try them out if you want!

In [28]:
print("\n".join([func for func in n.optimize.__dir__() if not func.startswith("_")]))

n
create_model
solve_model
assign_solution
assign_duals
post_processing
optimize_transmission_expansion_iteratively
optimize_security_constrained
optimize_with_rolling_horizon
optimize_mga
optimize_and_run_non_linear_powerflow
fix_optimal_capacities
fix_optimal_dispatch
add_load_shedding
